**HOMOLOGACION DE UNIVERSIDADES**

En resumen el problema se basa en poder encontrar la informacion de las universidades dadas por los postulantes (instituciones_educativas.csv) de manera que coincidan en lo mayor posible con las universidades licenciadas por SUNEDU (universidades.json) donde se encuentra la informacion correcta.

*1. El Primer paso es revisar los dataset y analizar si es que requiere una limpieza*

*2. Estimar cual es el porcentaje de obtener un mejor resultado al aplicar las transformaciones*

*3. Comparar el proceso crudo (sin mucha transformacion) vs Elaborando un proceso adecuado en obtener mayor información*

**1. Revisión y Analisis**

Se procedio a revisar los dataset, hay que tener ciertos puntos como criterio para el analisis:
    
*- Solo se deberia considerar Universidades, Se ha identificado Instituto y Colegio. Esto podria ayudar a no considerar dentro del analisis para obtener resultados mas precisos.*

*- Hay registros que presentan las siglas de alguna universidad, se podria analizar por separado dichos registros.*

*- Hay que considerar que en el analisis debemos considerar que se ignore la mayuscula y minuscula.*

**2. Estimar posibles resultados**

In [1]:
#Importamos las librerias que se usaran para este previo analisis
import numpy as np
import pandas as pd

#Almacenamos en variables los origenes de nuestros dataset

#Enlace del dataset instituciones_educativas.csv
url_csv = "https://raw.githubusercontent.com/nickiunac/homologar_universidades/main/instituciones_educativas.csv"

#Enlace del dataset Universidades.json
url_json = "https://raw.githubusercontent.com/nickiunac/homologar_universidades/main/Universidades.json"

#Leemos los archivos en los formatos correspondientes

#Dataframe de Instituciones Educativas
df_csv = pd.read_csv(url_csv)

#Dataframe de Universidades
df_json = pd.read_json(url_json)

*Revisamos que nuestros dataset no presenten vacios*

In [2]:
df_csv.isnull().sum()/len(df_csv)*100

candidateId    0.0
value          0.0
dtype: float64

In [3]:
df_json.isnull().sum()/len(df_json)*100

código INEI    0.0
Nombre         0.0
Siglas         0.0
dtype: float64

*Como el resultado mayor se obtiene de cuantos registros que tienen mayor coincidencia, nos centraremos en el dataset instituciones_educativas.csv donde se encuentra dicha informacion*

**Obtener 8588 registros como nuestro 100%**

In [4]:
#Mostramos la cantidad total de registros
df_csv.shape[0]

8588

*Ahora evaluaremos distintas formas de cual seria nuestro mayor margen en numero y/o porcentaje de obtener la mayor informacion correcta.*

*La mejor forma es analizar la palabra universidad en cuantos registros se presenta, eso seria el margen mas alto.*

**OJO:** *Hay que tener en cuenta que no todos esos registros van a ser universidades licenciadas, y considerar que puede haber registros con solamente siglas.*

In [5]:
#Analizamos cuantos registros contienen la palabara "universidad"

#Realizamos una copia para no alterar nuestro dataset
df_universidad = df_csv.copy()

#transformamos el campo a minuscula para evitar diferencias, luego buscamos la palabra "universidad"
df_universidad["value"] = df_universidad["value"].str.lower()
df_universidad = df_universidad[df_universidad["value"].str.contains("universidad")]

#Mostramos cuantos registros contienen la palabra "universidad"
df_universidad.shape[0]

2694

*Podemos concluir que nuestro margen mas alto seria **2694 Registros**, entonces deberiamos obtener un resultado cercano a ello para asi obtener la mayor informacion posible y menor error posible al transformar la información*

**3. Comparacion de procesos para obtener mayor resultado**

*Ahora realizaremos una transformacion con una herramienta util para la comparacion de cadenas de texto de manera difusa o aproximada que es la biblioteca de Python **fuzzywuzzy**, esta herramienta nos ayudara a calcular la similitud entre dos cadenas de texto.*

In [6]:
#Instalamos la biblioteca fuzzywuzzy

In [7]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [8]:
#Transformamos en minuscula los campos que realizaremos la comparacion y luego saber a que universidad licenciada pertenece
df_csv["value"] = df_csv["value"].str.lower()

#el campo figura con un espacio demas, es la forma como se obtiene del dataset pueden realizar una pequeña 
#transformacion "rename" o validan con un df_json.columns
df_json["Nombre "] = df_json["Nombre "].str.lower()

In [9]:
#Importamos las funciones
from fuzzywuzzy import fuzz, process

#Funcion para homologar universidades (para encontrar la mejor coincidencia)
#En este caso usamos el algoritmo "token_sort_radio" y realizamos una condicion
#donde se tiene que obtener una similitud mayor o igual a 90 y devuelve TRUE, sino devuelve FALSE
def homologar_universidad(value):
    match = process.extractOne(value, df_json['Nombre '], scorer=fuzz.token_sort_ratio)
    if match[1] >= 90:
        return True
    else:
        return False

#Creamos una columna de universidad homologada en df_csv para saber si pertenece a una universidad licenciada
df_csv['Universidad_Homologada'] = df_csv['value'].apply(homologar_universidad)

#Mostramos el resultado previo
df_csv.head()

C:\Users\Ricardo\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,candidateId,value,Universidad_Homologada
0,630633fa1e599d0009d6dee0,ucv,False
1,630633fa1e599d0009d6dee0,josé gálvez,False
2,63063bda29da5b00089fb224,universidad peruana de ciencias aplicadas - upc,False
3,63063bda29da5b00089fb224,centro peruano de estudios bancarios - cepeban,False
4,630644da1e599d0009d6e2f4,instituto de formación bancaria - ifb,False


In [10]:
#Contamos cuantos registros coincidentes obtuvimos
df_csv[(df_csv["Universidad_Homologada"] == True)].shape[0]

1635

**Entonces aplicando la herramienta fuzzywuzzy nos ayudo a obtener 1635 registros**

*Ahora vamos a ver cuando aplicamos un proceso de limpieza teniendo en cuenta nuestro analisis principal.*

**Generaremos un nuevo script "Proceso" donde visualizaremos todo el procedimiento, luego aqui compararemos los resultados**

*Ahora que visualizamos en el script **"Proceso"** se obtuvo 2202 registros*

**CONCLUSION**

In [11]:
#Creamos un pequeño dataframe con los datos obtenidos

#Estimago del margen mas alto al tener las coincidencias aproximadas de las universidades
Margen = 2694

data = {
    "Tipo": ["Proceso Simple", "Procesos Transformado"],
    "Registros": [1635, 2202]
}

resultado = pd.DataFrame(data)

resultado["%Total"] = np.divide(resultado["Registros"], Margen).round(2)

resultado.head()

,Tipo,Registros,%Total
0,Proceso Simple,1635,0.61
1,Procesos Transformado,2202,0.82


*Con la ayuda de la herramienta **fuzzywuzzy** y aplicando un analisis y transformacion logramos **incrementar más del *20% de similitud** con respecto al margen aproximado, esto ayudara a obtener mayor informacion y asi poder tomar una decision. sin embargo tener en cuenta que aun existen registros que pueden que no esten considerado o que se esten considerando mal pero con un minimo error, ya que se aplico una **similitud mayor o igual al 90**.*

**EN EL SCRIPT : PROCESO, ENCONTRARA TAMBIEN LAS SOLUCIONES DEL RETO, GENERANDO LOS DOS ARCHIVOS:**

        - universidades_homologadas.csv
        
        - sinonimo_universidades.json
        